In [1]:
# coding UTF-8
from input_m import read_file, conv_str_to_kana, conv_kana_to_vec, conv_vec_to_kana, conv_kana_to_vec_meta
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import numpy as np
import csv

data = read_file('dataset_proto.csv')
kana_title, kana_ans = conv_str_to_kana(data[0],data[1])
vec_title_vow, vec_title_con = conv_kana_to_vec(kana_title,1,"T")
vec_ans_vow, vec_ans_con = conv_kana_to_vec(kana_ans,1,"R")
vec_title = conv_kana_to_vec_meta(kana_title,1,"T")
vec_ans = conv_kana_to_vec_meta(kana_ans,1,"R")

"""
for i,kana_title in enumerate(kana_title):
    print(kana_title)
    print(vec_title[i])

clf = svm.SVC(gamma=0.001, C=100)
clf.fit(vec_title[:-1], vec_ans[:-1])

result = clf.predict(data[:-1])
print("実際の答え={0}, 予測結果={1}".format(vec_ans[-1], result))

result = clf.score(data, vec_ans)
print(result)
"""
# データセットを学習用とテスト用に分割
X_train, X_test, Y_train, Y_test = train_test_split(vec_title, vec_ans, train_size = 0.8, test_size = 0.2, random_state = 0)
X_train_vow, X_test_vow, y_train_vow, y_test_vow = train_test_split(vec_title_vow, vec_ans_vow, train_size = 0.8, test_size = 0.2, random_state = 0)
X_train_con, X_test_con, y_train_con, y_test_con = train_test_split(vec_title_con, vec_ans_con, train_size = 0.8, test_size = 0.2, random_state = 0)

"""
# 線形回帰に適用
lr = LinearRegression()
lr.fit(X_train, Y_train)
Y_pred = lr.predict(X_test)

Y_pred = np.array(Y_pred)
Y_test = np.array(Y_test)
"""

#アンサンブル学習
# train base model
base_model_1 = LinearRegression()
base_model_2 = LinearRegression()
#base_model_2 = LGBMRegressor()
#base_model_3 = KNeighborsRegressor()

base_model_1.fit(X_train_vow, y_train_vow)
base_model_2.fit(X_train_con, y_train_con)

# base predicts
base_pred_1 = base_model_1.predict(X_train_vow)
base_pred_2 = base_model_2.predict(X_train_con)

stacked_valid_predictions = np.column_stack((base_pred_1, base_pred_2))

meta_model = LinearRegression()
meta_model.fit(stacked_valid_predictions, Y_train)
#predicted_vector = base_pred_1 + base_pred_2
#meta_model.fit(predicted_vector,Y_train)
meta_valid_pred = meta_model.predict(X_test)

Y_pred = np.array(meta_valid_pred)
Y_test = np.array(Y_test)

#print ("mean squared error of model 1: {:.4f}".format(mean_squared_error(y_test_vow, base_pred_1)) )
#print ("mean squared error of model 2: {:.4f}".format(mean_squared_error(y_test_con, base_pred_2)) )
print ("mean absolute error of meta model: {:.4f}".format(mean_absolute_error(Y_test, meta_valid_pred)) )

# 予測と正解の差異を計算
pred_diff = []
for j,title in enumerate(Y_pred):
    title_diff = []
    for k,val in enumerate(title):
        diff = abs(val - Y_test[j,k])
        title_diff.append(diff)
    pred_diff.append(title_diff)

print(mean_absolute_error(Y_test, Y_pred))
# csvに出力
with open("pred_diff.csv", "w") as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(pred_diff)

# 0,1に変換
for title in Y_pred:
    for index,i in enumerate(title):
        if i<=0.35:
            title[index] = 0
        else:
            title[index] = 1
    # title[index] = round(j)

# カナに直して比較
Y_pred = Y_pred.tolist()
Y_test = Y_test.tolist()

Y_pred_kana = conv_vec_to_kana(Y_pred)
Y_test_kana = conv_vec_to_kana(Y_test)

for x, y in enumerate(Y_pred_kana):
    print(" ".join(Y_test_kana[x]))
    print(" ".join(y))
    print("\n")

ValueError: shapes (22,900) and (125,125) not aligned: 900 (dim 1) != 125 (dim 0)